In [1]:
# импорт необходимых библиотек

from math import asin
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

**Урок 8. Python применение статистических критериев на практике - пайплайн оценки A/B**
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue. Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [2]:
# Загрузим данные
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1IGMgJP_XqQwA6h_w1KYG7TQJhS15h4c4',
                                    dest_path='./gb_sem_8_cw.xlsx')

In [3]:
df = pd.read_excel('gb_sem_8_cw.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [6]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [7]:
# определим количсетво уникальных id
df.USER_ID.nunique()

6324

In [8]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'],
                as_index=False).agg({'REVENUE': 'sum'})
df

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.00
1,3,control,0.00
2,3,variant,0.00
3,4,variant,0.00
4,5,variant,0.00
...,...,...,...
7860,9996,control,0.00
7861,9996,variant,6.46
7862,9998,control,0.00
7863,10000,control,0.00


In [9]:
df.groupby('USER_ID', as_index=False).agg(
    {'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [10]:
# заменим variant_name на true/false индекс

unique_ids = \
    (df
     .groupby('USER_ID', as_index=False)
     .agg({'VARIANT_NAME': 'count'})
     .query('VARIANT_NAME == 1')
     .USER_ID
     .values
     )


In [11]:
df_changed = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [12]:
df_changed.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [13]:
# проверим данные на пропуски
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [14]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]

    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()

    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)

    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(
            n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(
            n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))

    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))

    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci,
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci, },
                          index=[column])
    return result

In [15]:
control = df_changed[df_changed['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_changed[df_changed['VARIANT_NAME'] == 'variant'].copy(deep=True)


continious_result(control, treatment, column='REVENUE')

100%|██████████| 10000/10000 [00:07<00:00, 1377.46it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.160587,0.712143,"[-0.004, 0.315]",0.121173


**ИТОГИ**
Результаты статистического анализа показывают отсутствие стат значимых изменений в выборках. Доверительный интервал [-0.005, 0.323 ] содержит 0, alpha и beta больше 5%.

Менеджеру следует внести другие изменения, чтобы получить статистически значимые различия в выборках.